In [2]:
#%%

import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np
from category_encoders import TargetEncoder
from sklearn.metrics import classification_report, accuracy_score
from sklearn.linear_model import LogisticRegression
from transformers import DistilBertTokenizer, DistilBertModel
import torch
import kagglehub

In [3]:


# Download the dataset
path = kagglehub.dataset_download("rtatman/blog-authorship-corpus")

print("Path to dataset files:", path)

# Optional: If the dataset includes a CSV file, load it into pandas
import os
import pandas as pd

# Assuming there's a CSV file in the downloaded path
for file in os.listdir(path):
    if file.endswith(".csv"):
        csv_path = os.path.join(path, file)
        df = pd.read_csv(csv_path)
        print(f"Loaded dataset from {csv_path}")
        break

# Example usage of the DataFrame
print(df.head())

Path to dataset files: C:\Users\91914\.cache\kagglehub\datasets\rtatman\blog-authorship-corpus\versions\2
Loaded dataset from C:\Users\91914\.cache\kagglehub\datasets\rtatman\blog-authorship-corpus\versions\2\blogtext.csv
        id gender  age              topic      sign          date  \
0  2059027   male   15            Student       Leo   14,May,2004   
1  2059027   male   15            Student       Leo   13,May,2004   
2  2059027   male   15            Student       Leo   12,May,2004   
3  2059027   male   15            Student       Leo   12,May,2004   
4  3581210   male   33  InvestmentBanking  Aquarius  11,June,2004   

                                                text  
0             Info has been found (+/- 100 pages,...  
1             These are the team members:   Drewe...  
2             In het kader van kernfusie op aarde...  
3                   testing!!!  testing!!!            
4               Thanks to Yahoo!'s Toolbar I can ...  


If for some reason the above code doesn't work, simply go to "https://www.kaggle.com/datasets/rtatman/blog-authorship-corpus", download the dataset and load using the next cell

In [ ]:
# %%

# df = pd.read_csv("archive/blogtext.csv")

In [4]:
# %%

df['age'].value_counts()

age
17    80859
24    80071
23    72889
16    72708
25    67051
26    55312
27    46124
15    41767
14    27400
34    21347
33    17584
35    17462
36    14229
13    13133
37     9317
38     7545
39     5556
40     5016
45     4482
43     4230
41     3738
48     3572
42     2908
46     2733
47     2207
44     2044
Name: count, dtype: int64

In [ ]:
#%%

# Clean the text data
def clean_text(text):
    # Remove URLs
    text = re.sub(r"http\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)
    # Remove HTML tags
    text = re.sub(r'<.*?>', '', text)
    # Remove non-alphanumeric characters and extra spaces
    text = re.sub(r'[^A-Za-z0-9\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [ ]:
#%%

df['clean_text'] = df['text'].apply(clean_text)
# Encode the gender column
label_encoder = LabelEncoder()
df['gender_encoded'] = label_encoder.fit_transform(df['gender'])  # Male: 0, Female: 1

In [ ]:
# %%

# Define the mapping
occupation_mapping = {
    # STEM
    'Technology': 'STEM', 'Engineering': 'STEM', 'Internet': 'STEM', 'Science': 'STEM',
    'Biotech': 'STEM', 'Telecommunications': 'STEM', 'Manufacturing': 'STEM', 
    'Chemicals': 'STEM', 'Architecture': 'STEM', 'Environment': 'STEM', 'Maritime': 'STEM',
    
    # Business, Finance, and Law
    'BusinessServices': 'Business-Finance-Law', 'Banking': 'Business-Finance-Law',
    'Accounting': 'Business-Finance-Law', 'InvestmentBanking': 'Business-Finance-Law',
    'RealEstate': 'Business-Finance-Law', 'Marketing': 'Business-Finance-Law', 
    'Advertising': 'Business-Finance-Law', 'Consulting': 'Business-Finance-Law',
    'Law': 'Business-Finance-Law', 'HumanResources': 'Business-Finance-Law',
    'Transportation': 'Business-Finance-Law',
    
    # Creative, Social, and Education
    'Arts': 'Creative-Social-Education', 'Education': 'Creative-Social-Education',
    'Religion': 'Creative-Social-Education', 'Communications-Media': 'Creative-Social-Education',
    'Publishing': 'Creative-Social-Education', 'Non-Profit': 'Creative-Social-Education',
    'Museums-Libraries': 'Creative-Social-Education', 'Fashion': 'Creative-Social-Education',
    'Sports-Recreation': 'Creative-Social-Education', 'Tourism': 'Creative-Social-Education',
    
    # Public Services and Miscellaneous
    'Government': 'Public-Services-Misc', 'Military': 'Public-Services-Misc',
    'LawEnforcement-Security': 'Public-Services-Misc', 'Construction': 'Public-Services-Misc',
    'Agriculture': 'Public-Services-Misc', 'Automotive': 'Public-Services-Misc',
    'Maritime': 'Public-Services-Misc', 'indUnk': 'Public-Services-Misc', 'Student': 'Public-Services-Misc'
}

# Apply the mapping
df['occupation_grouped'] = df['topic'].map(occupation_mapping)

# Check distribution
print(df['occupation_grouped'].value_counts())

occupation_grouped
Public-Services-Misc         420683
Creative-Social-Education    122837
STEM                          91538
Business-Finance-Law          46226
Name: count, dtype: int64


In [ ]:
# %%

# Define age groups
bins = [0, 17, 29, float('inf')]  # Adjust the ranges as per chosen grouping
labels = ['Adolescents', 'Young Adults', 'Adults']

# Create a new categorical column
df['age_group'] = pd.cut(df['age'], bins=bins, labels=labels, right=True)

# Check distribution
print(df['age_group'].value_counts())

age_group
Young Adults    321447
Adolescents     235867
Adults          123970
Name: count, dtype: int64


In [ ]:
# %%

# Encode the gender column
label_encoder = LabelEncoder()
df['gender_encoded'] = label_encoder.fit_transform(df['gender'])  # Male: 0, Female: 1

In [ ]:
# %%

# Encode the age_group column
label_encoder = LabelEncoder()
df['age_encoded'] = label_encoder.fit_transform(df['age_group'])

In [ ]:
# %%

# Split your data into training and testing
X_train, X_test, y_train, y_test = train_test_split(
    df[['clean_text','gender_encoded','occupation_grouped']],  # Replace 'other_features' with your actual columns
    df['age_encoded'],
    test_size=0.2,
    random_state=42
)

In [ ]:
#%%

print(f"Training set size: {len(X_train)}")
print(f"Test set size: {len(X_test)}")

Training set size: 545027
Test set size: 136257


In [ ]:
# %%

# Initialize the Target Encoder
encoder = TargetEncoder(cols=['occupation_grouped'])

# Fit the encoder on the training data and transform
X_train['occupation_encoded'] = encoder.fit_transform(X_train['occupation_grouped'], y_train)

# Apply the same transformation to the test data
X_test['occupation_encoded'] = encoder.transform(X_test['occupation_grouped'])

# Check results
print(X_train[['occupation_grouped', 'occupation_encoded']].head())

               occupation_grouped  occupation_encoded
278509  Creative-Social-Education            1.349294
572041       Public-Services-Misc            0.919386
146044       Public-Services-Misc            0.919386
569591       Public-Services-Misc            0.919386
580033                       STEM            1.543025


In [ ]:
#%%

X_train.drop(columns=['occupation_grouped'], inplace=True)

In [ ]:
#%%

X_test.drop(columns=['occupation_grouped'], inplace=True)

In [ ]:
#%%

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [ ]:
# %%

'''using Contextual embeddings'''

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')
model = model.to(device)
model.eval()

DistilBertModel(
  (embeddings): Embeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer): Transformer(
    (layer): ModuleList(
      (0-5): 6 x TransformerBlock(
        (attention): MultiHeadSelfAttention(
          (dropout): Dropout(p=0.1, inplace=False)
          (q_lin): Linear(in_features=768, out_features=768, bias=True)
          (k_lin): Linear(in_features=768, out_features=768, bias=True)
          (v_lin): Linear(in_features=768, out_features=768, bias=True)
          (out_lin): Linear(in_features=768, out_features=768, bias=True)
        )
        (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (ffn): FFN(
          (dropout): Dropout(p=0.1, inplace=False)
          (lin1): Linear(in_features=768, out_features=3072, bias=True)
          (lin2): Li

In [ ]:
#%%

import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import classification_report, accuracy_score
from tqdm import tqdm

In [ ]:
# %%

def generate_embeddings(texts, tokenizer, model, max_length=128, batch_size=32):
    embeddings = []
    for i in tqdm(range(0, len(texts), batch_size), desc="Generating Embeddings"):
        # Tokenize text
        batch_texts = texts[i:i+batch_size]
        tokens = tokenizer(
            batch_texts,
            padding='max_length',
            truncation=True,
            max_length=max_length,
            return_tensors="pt"
        )
        # print(tokens)
        input_ids = tokens['input_ids'].to(device)
        # print(input_ids)
        attention_mask = tokens['attention_mask'].to(device)
        # print(attention_mask)

        # Pass through model
        with torch.no_grad():
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            # print(outputs)
            token_embeddings = outputs.last_hidden_state  # Shape: [batch_size, sequence_length, hidden_size]
            sentence_embedding = torch.mean(token_embeddings, dim=1)  # Mean pooling over tokens
            # print(sentence_embedding)
            embeddings.append(sentence_embedding)
    
    return torch.cat(embeddings, dim=0)  # Return as a single tensor

In [ ]:
#%%

# Generate embeddings for training and testing datasets
X_train_embeddings = generate_embeddings(X_train['clean_text'].tolist(), tokenizer, model)

Generating Embeddings: 100%|██████████| 17033/17033 [28:43<00:00,  9.88it/s]


In [ ]:
#%%

occupation_train_tensor = torch.tensor(X_train['occupation_encoded'].values, dtype=torch.float32).unsqueeze(1).to(device)
gender_train_tensor = torch.tensor(X_train['gender_encoded'].values, dtype=torch.float32).unsqueeze(1).to(device)
X_train_combined = torch.cat([X_train_embeddings, occupation_train_tensor, gender_train_tensor], dim=1)

print(f"Combined Train Features Shape: {X_train_combined.shape}")

Combined Train Features Shape: torch.Size([545027, 770])


In [ ]:
#%%

X_test_embeddings = generate_embeddings(X_test['clean_text'].tolist(), tokenizer, model)

Generating Embeddings: 100%|██████████| 4259/4259 [07:08<00:00,  9.95it/s]


In [ ]:
#%%

occupation_test_tensor = torch.tensor(X_test['occupation_encoded'].values, dtype=torch.float32).unsqueeze(1).to(device)
gender_test_tensor = torch.tensor(X_test['gender_encoded'].values, dtype=torch.float32).unsqueeze(1).to(device)
X_test_combined = torch.cat([X_test_embeddings, occupation_test_tensor, gender_test_tensor], dim=1)

print(f"Combined Test Features Shape: {X_test_combined.shape}")

Combined Test Features Shape: torch.Size([136257, 770])


In [ ]:
# %%

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train_combined, dtype=torch.float32).to(device)
X_test_tensor = torch.tensor(X_test_combined, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long).to(device)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long).to(device)

<ipython-input-27-c0daa6752426>:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train_tensor = torch.tensor(X_train_combined, dtype=torch.float32).to(device)
<ipython-input-27-c0daa6752426>:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test_tensor = torch.tensor(X_test_combined, dtype=torch.float32).to(device)


In [ ]:
# %%

# Define the model
class AgeGroupPredictor(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(AgeGroupPredictor, self).__init__()
        self.fc1 = nn.Linear(input_dim, 256)
        self.fc2 = nn.Linear(256,128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 32)
        self.fc5 = nn.Linear(32, num_classes)
        self.dropout = nn.Dropout(0.3)
        self.relu = nn.ReLU()
        # self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.relu(self.fc3(x))
        x = self.relu(self.fc4(x))
        x = self.fc5(x)
        return x

In [ ]:
# %%

# Initialize model
input_dim = X_train_combined.shape[1]
num_classes = len(y_train.unique())  # Number of classes
model = AgeGroupPredictor(input_dim, num_classes).to(device)

In [ ]:
# %%

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()  # Binary Cross Entropy Loss
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# %%

# Training loop
epochs = 50
batch_size = 32
model.train()

for epoch in range(epochs):
    epoch_loss = 0
    correct=0
    total = 0
    print(f"Epoch {epoch+1}/{epochs}")
    for i in tqdm(range(0, len(X_train_tensor), batch_size), desc="Training Batches"):
        # Batch data
        batch_X = X_train_tensor[i:i+batch_size]
        batch_y = y_train_tensor[i:i+batch_size]
        
        # Forward pass
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        epoch_loss += loss.item()
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Calculate training accuracy
        predicted = torch.argmax(outputs, dim=1)
        correct += (predicted == batch_y).sum().item()
        total += batch_y.size(0)
    
    # Calculate metrics
    avg_loss = epoch_loss / (len(X_train_tensor) // batch_size)
    accuracy = correct / total * 100
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {avg_loss:.4f}, Accuracy: {accuracy:.2f}%")

# Evaluate the model
model.eval()
with torch.no_grad():
    outputs = model(X_test_tensor)
    y_pred = torch.argmax(outputs, dim=1)

print(f"Test Accuracy: {accuracy_score(y_test_tensor.cpu(), y_pred.cpu()):.4f}")
print("\nClassification Report:")    
print(classification_report(y_test_tensor.cpu(), y_pred.cpu()))

Epoch 1/50


Training Batches: 100%|██████████| 17033/17033 [00:37<00:00, 452.15it/s]


Epoch [1/50], Loss: 0.8209, Accuracy: 61.95%
Epoch 2/50


Training Batches: 100%|██████████| 17033/17033 [00:35<00:00, 477.55it/s]


Epoch [2/50], Loss: 0.7944, Accuracy: 63.32%
Epoch 3/50


...

Epoch [48/50], Loss: 0.7210, Accuracy: 67.45%
Epoch 49/50


Training Batches: 100%|██████████| 17033/17033 [00:36<00:00, 465.59it/s]


Epoch [49/50], Loss: 0.7212, Accuracy: 67.37%
Epoch 50/50


Training Batches: 100%|██████████| 17033/17033 [00:36<00:00, 471.10it/s]


Epoch [50/50], Loss: 0.7205, Accuracy: 67.42%
Test Accuracy: 0.6673

Classification Report:
              precision    recall  f1-score   support

           0       0.73      0.73      0.73     47221
           1       0.68      0.19      0.30     24592
           2       0.63      0.80      0.71     64444

    accuracy                           0.67    136257
   macro avg       0.68      0.58      0.58    136257
weighted avg       0.67      0.67      0.64    136257



In [ ]:
X_train_combined

tensor([[ 0.1233,  0.2310,  0.0882,  ..., -0.1283,  1.3493,  1.0000],
        [ 0.0530,  0.0098,  0.1782,  ...,  0.0212,  0.9194,  0.0000],
        [ 0.1453,  0.0833,  0.4302,  ..., -0.1145,  0.9194,  0.0000],
        ...,
        [ 0.0402, -0.0676,  0.2227,  ..., -0.0413,  0.9194,  0.0000],
        [-0.0404,  0.1895,  0.2807,  ...,  0.0770,  1.3493,  0.0000],
        [-0.1729,  0.1082,  0.3781,  ..., -0.0182,  1.3493,  1.0000]],
       device='cuda:0')

In [ ]:
X_train_combined.shape

torch.Size([545027, 770])

In [ ]:
#%%

torch.save(X_train_embeddings, "X_train_embeddings.pt")
torch.save(X_test_embeddings, "X_train_embeddings.pt")

In [ ]:
#%%

torch.save(X_train_embeddings, "X_train_embeddings.pt")

In [ ]:
#%%

torch.save(X_test_embeddings, "X_test_embeddings.pt")